# init

In [ ]:
import json
import pickle
import uvicorn
import numpy as np
from datetime import datetime

import nest_asyncio
nest_asyncio.apply()

In [ ]:
PORT = 8000

In [ ]:
class ModelPredictor:
    """this class is body persing and predict.

    attributes:
        model(:obj:): model.

    """
    
    def __init__(self):
        self.model = pickle.load(open('model', 'rb'))

    # async def predict(self,receive):
    def predict(self,body):
        """predict from recieve data
    
        predict from recieve data
    
        Args:
           recieve(obj:): http recieved object
    
        Returns:
            bytes: byte object

        Examples:
            >>> none
        """
        rt_val = self.model.predict([[
                    body['sepal length (cm)'],
                    body['sepal width (cm)'],
                    body['petal length (cm)'],
                    body['petal width (cm)']]])
        return json.dumps({"val": rt_val[0]}, cls=NumpyEncoder).encode()

In [ ]:
class NumpyEncoder(json.JSONEncoder):
    """numpy data encode

    attributes:
        model(:obj:): model.

    """


    def default(self, obj):
        """validate object
    
        Args:
           obj(np.object): numpy object.
    
        Returns:
            (obj: int | float | list): object

        Examples:
            >>> none
        """
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NumpyEncoder, self).default(obj)

In [ ]:
prd_controller = ModelPredictor()

In [ ]:
prd_controller.predict({"sepal length (cm)": 1,"sepal width (cm)": 1,"petal length (cm)": 1,"petal width (cm)":1})

# create prediction server

In [ ]:
async def read_body(receive):
    """
    Read and return the entire body from an incoming ASGI message.
    """
    body = b''
    more_body = True

    while more_body:
        message = await receive()
        body += message.get('body', b'')
        more_body = message.get('more_body', False)

    return json.loads(body)


async def app(scope, receive, send):
    """
    Echo the request body back in an HTTP response.
    """
    body = await read_body(receive)
    ret = prd_controller.predict(body)
    await send({
        'type': 'http.response.start',
        'status': 200,
        'headers': [
            [b'content-type', b'text/plain'],
        ]
    })
    await send({
        'type': 'http.response.body',
        'body': ret,
    })

In [ ]:
uvicorn.run(app, host="0.0.0.0", port=PORT, log_level="info")